In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import pymongo
import re
import pymysql
from sqlalchemy import create_engine
pymysql.install_as_MySQLdb()
import time

In [33]:
# getting gkey from the config file
from config import gkey

# getting the info for sql bench
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [34]:
# Creating engine to mysql
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

In [35]:
#results[8]['photos'][0]['photo_reference']

In [36]:
def generate_results_dataframe(results):
    results_list = []

    for r in results:
        test = True

        # Initialize variables
        business_status = None
        lat = None
        lng = None
        icon = None,
        name = None
        place_id = None
        price_level = None
        photo_reference = None
        rating = None
        place_types = None
        user_ratings_total = None
        address = None

        try:
            if 'business_status' in r.keys(): business_status = r['business_status']
        except: pass

        try:
            if 'lat' in r['geometry']['location'].keys(): lat = r['geometry']['location']['lat']
        except: pass
        try:
            if 'lng' in r['geometry']['location'].keys(): lng = r['geometry']['location']['lng']
        except: pass

        try:
            if 'icon' in r.keys(): icon = r['icon']
        except: pass

        try:
            if 'name' in r.keys(): name = r['name']
        except: pass

        try:
            if 'place_id' in r.keys(): place_id = r['place_id']
        except: pass

        try:
            if 'price_level' in r.keys(): price_level = r['price_level']
        except: pass

        try:
            if 'photos' in r.keys(): photo_reference = r['photos'][0]['photo_reference']
        except: pass

        try:
            if 'rating' in r.keys(): rating = r['rating']
        except: pass

        try:
            if 'types' in r.keys(): place_types = r['types']
        except: pass

        try:
            if 'user_ratings_total' in r.keys(): user_ratings_total = r['user_ratings_total']
        except: pass

        try:
            if 'vicinity' in r.keys(): address = r['vicinity']
        except: pass

        result_dict = {
            'city': city_name,
            'type': biz_type,
            'keyword': keyword,
            'business_status': business_status,
            'lat': lat,
            'lng': lng,
            'icon': icon,
            'name': name,
            'place_id': place_id,
            'price_level': price_level,
            'photo_reference': photo_reference,
            'rating': rating,
            'place_types': place_types,
            'user_ratings_total': user_ratings_total,
            'address': address
        }

        results_list.append(result_dict)

    results_df = pd.DataFrame(results_list)
    results_df['place_types'] = results_df['place_types'].astype(str)
    
    return results_df

In [37]:
def load_into_database(response):

    while('results' in response.keys() and len(response['results']) > 0):   

        results = response['results']

        # call generate_results_dataframe function and pass the API response results to it
        # this will return a DataFrame that contains the data returned by the API
        results_df = generate_results_dataframe(results)

        # post to SQL
        # PLEASE DO NOT WRITE TO THE restraurants TABLE!!!
        # Write to your own table by place a suffix behind restaurants
        results_df.to_sql('restaurants_tbl', con=conn, if_exists='append', index=False)

        print(f'--- {len(results_df)} record(s) loaded to the database --- ')

        if 'next_page_token' in response:      
            pagetoken = response['next_page_token']

            # sleeping for the "short delay" 
            time.sleep(10)

            base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
            params = {
                "pagetoken": pagetoken,
                "key": gkey,
            }

            response = requests.get(base_url, params=params).json()

        else:
            break

In [38]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

San_Juan = ['18.466333, -66.105721']
Ponce = ['18.01108, -66.61406']
Rincon = ['18.340216, -67.250015']
Fajardo = ['18.325787, -65.652382']
Arecibo = ['18.47245, -66.71573']
Humacao = ['18.148750, -65.819099']
Mayaguez = ['18.201349, -67.139488']

city_coords = [san_juan, Ponce, rincon, fajardo, arecibo, humacao, mayaguez]

city = ["San Juan", "Ponce", "Rincon", "Fajardo", "Arecibo", "Humacao", "Mayaguez"]

keywords = ["bar restaurant", "chinese",  "fast food", "japanese", "middle eastern", "puertorican", "spanish"]
# keywords = ["japanese"]

biz_type = "restaurant"

i = 0
for coords in city_coords:
    city_name = city[i]
        
    for keyword in keywords:
        print(f'Processing {city_name}  {biz_type}    {keyword}')
        
        params = {
            "location": coords,  # coords
            "radius": 8000,
            "types": biz_type,
            "key": gkey,
            "keyword": keyword
        }

    # Initial API call
        response = requests.get(base_url, params=params).json()
        print(response)
#         break
    
        load_into_database(response)
        
    i += 1
#     break

Processing San Juan  restaurant    japanese
{'html_attributions': [], 'next_page_token': 'ATtYBwLXBuF-K7TKf0bEv_Bnv8OOPSBfXyZr6g7qR5qI_YcN9QABFQIC5nyCEaV1Cow44Y7cOW6MT7pv0Petj4FKMNs-iYofNEQ_TDohy3OJcn7IEIkbfeffexunv9I2T6NX_LUVzkA_NrMKXCCp8wRp8jvcN7rBpu_bwY27RfiWRvTeT28QuJ6_V_0xGFhXiZW-uNeig14PhTv4U27K-RF7U_GgeaVjn4fao1yM6AFeWQEQrectXaPI-jWBQXLy1OliiwTgnK3RXzjqDEB3kGvCmslap6Ex-vfLPVP81B6ijGciJnOn8q0FKBFobfqvgs08YntogdSfmKz_EtoaQVEq2H5D_unens378KXwCSSg5UgkSrgXgFpOJalKV9YnhBMW6wnGtQMpdEW6aRYZYak3SNtSpBYGiZobn4iVHR9i325V-ilDELE1nglfKnYkNlIvJ-6uvoUeE0z6okEkBcyPhXWDDJ4', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 18.4564255, 'lng': -66.0724512}, 'viewport': {'northeast': {'lat': 18.45772282989272, 'lng': -66.07112187010729}, 'southwest': {'lat': 18.45502317010728, 'lng': -66.07382152989273}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png', 'name': 'Nagoya Sushi & Tiki Bar Condado', 'opening_hours': {'open_now': Tr

--- 20 record(s) loaded to the database --- 
--- 15 record(s) loaded to the database --- 
Processing Ponce  restaurant    japanese
{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 17.9722624, 'lng': -66.5940866}, 'viewport': {'northeast': {'lat': 17.97382702989272, 'lng': -66.59274352010728}, 'southwest': {'lat': 17.97112737010728, 'lng': -66.59544317989273}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png', 'name': "Sancho's", 'opening_hours': {'open_now': False}, 'photos': [{'height': 981, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/106563173925059067389">Jesús García</a>'], 'photo_reference': 'ATtYBwLucYvd0LFMyzVh96wiMMMtpj0uCje18HTkBX96gIcyV4hxNDCSVnzJHYkoa8SiV4rLTE7ED1ls8KiP-pfUCE_OEn3eAh9-XtMTj2NvdtI7lPJxj5EVC8blXxctygrt2LA7X86wMrPDuTR4bm2YuoSp0vbzfVGBz8HokSqvmQtCAP_P', 'width': 1743}], 'place_id': 'ChIJswW1_g7WHIwRhrG-FaEFcuw', 'plus_code': {'compound_code':

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 18.3454403, 'lng': -65.66999419999999}, 'viewport': {'northeast': {'lat': 18.34702997989272, 'lng': -65.66859642010728}, 'southwest': {'lat': 18.34433032010728, 'lng': -65.67129607989271}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png', 'name': 'GM Sushi Bar', 'opening_hours': {'open_now': False}, 'photos': [{'height': 3024, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/109089007971944072944">Amyvette Herrera</a>'], 'photo_reference': 'ATtYBwJwyuS-e6LjYHaKvN3HaHnzXmfRVniFAYKKpVCPuWSWhXrvRm9H2O9QXCas_hFUd0l2lpIYpikot4E57h19Kz55ElfcHr8xtq74Hp1qICXkQDMHXYmYYJuMVjy6H-DNwcmqoSTqmOSzYWsrBjE_uCK7a5Y7AzqsB9a9SYeR0VDULx2o', 'width': 4032}], 'place_id': 'ChIJSzGDytSZBIwRmVL9HFNnsxA', 'plus_code': {'compound_code': '88WJ+52 Fajardo', 'global_code': '77CP88WJ+52'}, 'rating': 4.4, 'reference': 'ChIJSzGDytSZBIwRmVL9HFNnsxA', 'scop

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 18.1557753, 'lng': -67.1458817}, 'viewport': {'northeast': {'lat': 18.15711792989272, 'lng': -67.14453387010728}, 'southwest': {'lat': 18.15441827010728, 'lng': -67.14723352989272}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png', 'name': 'Ginza Sushi Japanese & Oriental Cuisine Restaurant', 'opening_hours': {'open_now': False}, 'photos': [{'height': 3120, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/101067996380375956647">Abner Hernandez</a>'], 'photo_reference': 'ATtYBwI22wDgbqXH1euDnu2S-6dCuHNxHmmvku0CaGJgGbnLZyU0PtHlRaD064k4bSJQo5899JUkpDSz_fOMi8sYkdZ8yhyowWozFtgQ2FDDt4VHXS2YLlA_xyhWTl4KLXdf6vTCbVm3Oi23HcTdHkrLA7DEmKdsllEWNWTLpG5Lcc4crTBq', 'width': 4160}], 'place_id': 'ChIJKxVqrRWzAowRsomRXl7IDtw', 'plus_code': {'compound_code': '5V43+8J Mayagüez', 'global_code': '77CJ5V43+8J'}, 'rating': 4.7, 'reference': 'ChIJ